In [1]:
from model import ChoiceModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
from tqdm import tqdm
import numpy as np
import random  

np.random.seed(42)
random.seed(42)

In [2]:
def process_row(choice_model, row):
    profile = row[['person_id', 'age', 'individual_income',
                    'household_size', 'family_structure', 'vehicles', 'cypher']]
    profile = profile.to_dict()
    response = choice_model.infer(
        profile=profile, mode='experiment',in_parallel=True)
    return response

def run_experiments(desire,num_sample,num_threads = 5):
    print("get test data")
    test_data_path = f'data/test/{desire}.csv'
    test_df = pd.read_csv(test_data_path, index_col=False)
    print(f"start experiemtns (desire={desire},sample_num={num_sample})")
    choice_model = ChoiceModel(
    data_dir='data', desire=desire, sample_num=num_sample, skip_init=False)
    data = []
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_row, choice_model, row) for idx, row in test_df.iterrows()]
        for future in tqdm(as_completed(futures), total=len(futures)):
            try:
                response = future.result()
                if response is not None:
                    data.append(response)
            except Exception as e:
                print(f"Error get response: {e}")

    data_df = pd.DataFrame(data, columns=['person_id', 'profile', 'top_k', 'desire', 'city', 'cypher',
                                      'amenity_recommendation', 'amenity_llm_choice', 'amenity_final_choice',
                                      'mode_recommendation', 'mode_llm_choice', 'mode_final_choice'])
    data_df.to_csv(choice_model.log_data_path)
    print(f'data saved to {choice_model.log_data_path}')
    print("=="*20)

In [3]:
run_experiments(desire='Eat',num_sample=1100,num_threads = 5)

get test data
start experiemtns (desire=Eat,sample_num=1100)


adding go_to edges...: 100%|██████████| 1100/1100 [00:03<00:00, 327.59it/s]


creating neo4j index...
done!


100%|██████████| 1000/1000 [27:58<00:00,  1.68s/it]

data saved to data/logs/1100/Eat.csv
